<a href="https://colab.research.google.com/github/bdi2357/CoverLetter/blob/main/CoverLetrrer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
!git clone https://github.com/bdi2357/CoverLetter

Cloning into 'CoverLetter'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 19), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 114.94 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
!pip install -q PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00


In [5]:
!pip install docx
!pip install fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=335b190df11f9da707275e4d4b4e988bd17e0ccc6419bb34251b0d6b5b1f4919
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=8ca185898240caccc85cdd3853cc2fa17b7a77817714ffcb03a2bf9faa252009
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [6]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.3 MB/s eta 0:00:00


In [7]:
#from genai import GenerativeModel
from docx import Document
from docx.shared import Pt
import re
import time

In [8]:
# Used to securely store your API key
from google.colab import userdata

In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [12]:
from PyPDF2 import PdfReader

def load_and_extract_text(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

In [13]:
def generate_cover_letter(model,cv_text, job_description_text):

    prompt = f"""Based on the provided job description and CV, write a concise, compelling and personalized cover letter that highlights relevant skills and experiences.

    Job Description:\n{job_description_text}\n\nCV:\n{cv_text}\n\n

    Tailor the letter to the specific job requirements and showcase the candidate's match for the position. Be sure to mention specific accomplishments and quantify results whenever possible. Keep the tone professional and precise based on the cv.
    As concise as possible no more than 4 sentences. Avoid using adjectives unless neccessary , as less as possible , do not use emotive adjectives in any case, use only factual information based on the cv_text , finalize with best regards, Name where the name of the applicant is taken from the CV"""
    print("HERE generate_cover_letter")
    response = model.generate_content(prompt)  # Remove the 'context' argument
    print("after generate_cover_letter")
    return response

In [14]:
job_description = """ We're seeking an AI Developer to join our team. In this role, you'll leverage artificial intelligence and machine learning techniques to improve the invoice reconciliation process and create a unified data format across various financial systems.

Responsibilities:
- Develop and implement AI and machine learning models to automate invoice reconciliation
- Create intelligent systems to unify diverse invoice data into a standardized format
- Design and build natural language processing (NLP) solutions to extract key information from unstructured invoice data
- Implement machine learning algorithms to identify patterns, anomalies, and potential errors in financial data
- Collaborate with finance and accounting teams to understand business requirements and integrate AI solutions into existing workflows
- Continuously improve and optimize AI models based on new data and changing business needs

Requirements:
- Masters or PhD in Computer Science, Artificial Intelligence, or related field
- 3+ years of experience developing AI and machine learning solutions, preferably in finance or accounting domains
- Strong programming skills in Python and experience with ML frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
- Experience with NLP techniques and text analysis
- Familiarity with financial systems, ERP software, and accounting principles
- Knowledge of data privacy and security best practices
- Excellent problem-solving skills and ability to translate complex business requirements into technical solutions

No agencies please.
This individual must be available for 30h/week and during UK working hours.

"""

In [15]:
cv_text = load_and_extract_text("CoverLetter/Data/CV_GPT_rev.pdf")

In [56]:
cv_text

'Itay\nBen-Dan\nHaarava\n20,\nHerzliya\nP.\nO.\nBox\n5990,\nHerzliya,\nIsrael\n46100\nCellular:\n+972544539284\nEmail:\nitaybd@gmail.com\nProfessional\nSummary\nFull\nStack\nMachine\nLearning\nEngineer\nwith\nextensive\nexperience\nin\ndata\nengineering,\nmodel\ndevelopment,\ndeployment,\nand\nmaintenance.\nExpertise\nin\nsports,\nfinance,\nand\nrobotics,\nwith\na\nfocus\non\nadvanced\ndata\nanalytics\nand\nmachine\nlearning,\nincluding\nLLMs\nand\nmulti-agent\nsystems.\nWork\nExperience\n2017-Present:\nMachine\nLearning\nand\nData\nScience\nConsultant\n●\nDeveloped\npredictive\nmodels\nfrom\nscratch,\nincluding\ndeployment.\n●\nConducted\nexploratory\ndata\nanalysis\nand\nadvanced\nfeature\ngeneration.\n●\nIntegrated\ndata\nfrom\nvarious\nsources\ninto\nunified\nformats.\n●\nDeveloped\npolicy\ngenerative\nmodels\nusing\nreinforcement\nlearning.\n2017-Present:\nFounder\nand\nHead\nof\nResearch,\nFinzor\nLtd.\n●\nDeveloped\nportfolio\nmanagement\nand\ninvestment\nanalysis\ntools.\n●\nLe

In [16]:
generate_cover_letter(model,cv_text, job_description).text

HERE generate_cover_letter
after generate_cover_letter


'Dear Hiring Manager,\n\nI am writing to express my interest in the AI Developer position at your company. My extensive experience in developing and deploying AI and machine learning solutions, particularly in the finance and accounting domains, aligns well with your requirements. I have a proven track record of success in projects involving data engineering, model development, and integration of AI solutions into existing workflows. I am confident that my skills and experience would be a valuable asset to your team. \n\nBest regards,\n\nItay Ben-Dan \n'

In [17]:
def extract_key_phrases(model, cover_letter_text, job_description_text, num_phrases=4):
    """
    Extracts key phrases from a cover letter based on their relevance to a job description using Gemini.

    Args:
        model: The Gemini language model instance.
        cover_letter_text: The text of the cover letter.
        job_description_text: The text of the job description.
        num_phrases: The number of phrases to extract (default is 4).

    Returns:
        A list of the most important phrases.
    """

    prompt = f"""Given the following cover letter and job description, identify the {num_phrases} most important words or two-word expressions in the cover letter that are most relevant to the job description.
    These words or Two words expression should show professional skills of the candidate they should reflecy the candidate as a doer, if it is an action like developing/programming it should be accompanied with the subject of the action. such as :
    developing AI models.

    Cover Letter:
    {cover_letter_text}

    Job Description:
    {job_description_text}
    """
    print("HERE key phrases")
    response = model.generate_content(prompt)

    # Extract key phrases from response
    important_phrases = response.text.split('\n')  # Assuming each phrase is on a new line
    print(len(important_phrases))
    return important_phrases

In [20]:
time.sleep(3)
job_description = job_description #load_and_extract_text("job_description.pdf")
cv_text = load_and_extract_text("CoverLetter/Data/CV_GPT_rev.pdf")
cover_letter_text = generate_cover_letter(model,cv_text, job_description).text
"""
for k in cover_letter_text.split("."):
  print(k)
time.sleep(3)
key_phrases = extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3)
for k in key_phrases:
  print("#"*30)
  print(k)
"""

HERE generate_cover_letter
after generate_cover_letter


'\nfor k in cover_letter_text.split("."):\n  print(k)\ntime.sleep(3)\nkey_phrases = extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3)\nfor k in key_phrases:\n  print("#"*30)\n  print(k)\n'

In [18]:
def extract_strings(key_phrases):
  R= []
  for k in key_phrases:
    A = (re.findall('\*\*.*?\*\*',k))
    if len(A)>0:
      R += re.findall('[A-Za-z\ 0-9]+',A[0])
  return R

In [21]:
extract_strings(extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3))

HERE key phrases
6


['Developing AI models', 'Integrating diverse data', 'NLP techniques']

In [22]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

def bold_strings_in_text(text, bold_strings, output_doc):
    # Create a new Document
    doc = Document()

    # Add the text to the document, with specified strings in bold
    para = doc.add_paragraph()
    parts = split_text(text, bold_strings)

    for part in parts:
        if part.lower() in [s.lower() for s in bold_strings]:
            run = para.add_run(part)
            run.bold = True
        else:
            para.add_run(part)

    # Save the document
    doc.save(output_doc)

def split_text(text, bold_strings):
    import re
    pattern = '|'.join(re.escape(s) for s in bold_strings)
    parts = re.split(f'({pattern})', text)
    print(parts)
    return parts

# Example usage
text = "This is a sample text. We want some words like sample and words to be bold."
bold_strings = ["sample", "words"]
bold_strings_in_text(text, bold_strings,"output.docx")



['This is a ', 'sample', ' text. We want some ', 'words', ' like ', 'sample', ' and ', 'words', ' to be bold.']


In [23]:
cl_text = generate_cover_letter(model,cv_text, job_description).text
strings = extract_strings(extract_key_phrases(model, cl_text, job_description, num_phrases=3))
bold_strings_in_text(cl_text, strings, "test2.docx")

HERE generate_cover_letter
after generate_cover_letter
HERE key phrases
6
['Dear Hiring Manager,\n\nI am writing to express my strong interest in the AI Developer position at your company. My extensive experience in developing and deploying AI solutions, particularly ', 'in finance and data analytics', ", aligns well with your requirements. My expertise in Python, machine learning frameworks, and NLP techniques, combined with my experience in building and integrating AI systems into financial workflows, make me a suitable candidate.  I am particularly interested in the opportunity to leverage my skills in invoice reconciliation and data unification, and I am eager to contribute to your team's success. \n\nBest Regards,\n\nItay Ben-Dan\n"]


In [24]:
print(strings)

['Developing AI solutions ', 'in finance and data analytics', 'Building and integrating AI systems into financial workflows', 'Leveraging skills in invoice reconciliation and data unification']


In [65]:
cl_text

"Dear Hiring Manager,\n\nMy experience as a Machine Learning Engineer with extensive expertise in developing and deploying AI solutions, particularly in finance, aligns well with the requirements for this role. I have successfully developed predictive models, integrated diverse data into unified formats, and applied NLP techniques for data extraction. I am confident in my ability to contribute significantly to your team's efforts to automate invoice reconciliation and optimize financial processes. \n\nBest Regards, \nItay Ben-Dan \n"

In [66]:
!ls

CoverLetter  output.docx  output_professional.pdf  sample_data	test2.docx


In [25]:
!pip install python-docx reportlab pypandoc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.4 MB/s eta 0:00:00


In [26]:
from docx import Document
import re
import pypandoc
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [27]:
!pip install docx2pdf

In [28]:
from docx import Document
import re
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from io import BytesIO


In [29]:
def convert_docx_to_pdf(input_docx, output_pdf, title, name):
    # Convert docx to PDF using BytesIO to avoid creating temporary files
    pdf_buffer = BytesIO()
    pdf = canvas.Canvas(pdf_buffer, pagesize=letter)
    width, height = letter

    # Add title and name
    pdf.setFont("Helvetica-Bold", 16)
    pdf.drawString(72, height - 72, title)
    pdf.setFont("Helvetica", 12)
    pdf.drawString(72, height - 92, name)

    # Read the docx file and add content to the PDF
    doc = Document(input_docx)
    text_object = pdf.beginText(72, height - 112)
    text_object.setFont("Helvetica", 10)

    for para in doc.paragraphs:
        for run in para.runs:
            if run.bold:
                text_object.setFont("Helvetica-Bold", 10)
            else:
                text_object.setFont("Helvetica", 10)
            text_object.textLine(run.text)

    pdf.drawText(text_object)
    pdf.save()

In [30]:
name = "Itay Ben-Dan"
title = "AI Developer Application"
cl_pdf_name = "output.docx"
output_doc_name = "cl_output.pdf"
convert_docx_to_pdf(cl_pdf_name, output_doc_name, title, name)

In [31]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
import re

def create_pdf_with_bold_strings(output_pdf, title, name, text, bold_strings):
    # Create the PDF document
    pdf = SimpleDocTemplate(output_pdf, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Define custom styles for title and body text
    title_style = ParagraphStyle(name='Title', fontSize=16, leading=20, spaceAfter=20, alignment=1)
    name_style = ParagraphStyle(name='Name', fontSize=12, leading=14, spaceAfter=14, alignment=1)
    body_style = styles['BodyText']
    bold_style = ParagraphStyle(name='Bold', parent=styles['BodyText'], fontName='Helvetica-Bold')

    # Add title
    story.append(Paragraph(title, title_style))

    # Add name
    story.append(Paragraph(name, name_style))
    story.append(Spacer(1, 12))

    # Process and add the main text with bold strings
    parts = split_text(text, bold_strings)
    for part in parts:
        if any(re.fullmatch(re.escape(b), part, re.IGNORECASE) for b in bold_strings):
            story.append(Paragraph(part, bold_style))
        else:
            story.append(Paragraph(part, body_style))

    # Build the PDF
    pdf.build(story)

def split_text(text, bold_strings):
    pattern = '|'.join(re.escape(s) for s in bold_strings)
    parts = re.split(f'({pattern})', text, flags=re.IGNORECASE)
    return parts

# Example usage
cv_text = "John Doe"
text = """Dear Hiring Manager,

I am writing to express my strong interest in the AI Developer position at your company. My extensive experience in developing and deploying AI solutions, particularly in finance and data analytics, aligns well with your requirements. My accomplishments include building predictive models from scratch, integrating data from diverse sources into unified formats, and creating adaptive models for updating anti-malware filters. I am confident that my skills and experience would be a valuable asset to your team.
"""
bold_strings = ["developing and deploying AI solutions", "integrating data from diverse sources", "creating adaptive models"]
output_pdf = "output_professional.pdf"
title = "AI Developer Application"

create_pdf_with_bold_strings(output_pdf, title, cv_text, text, bold_strings)


In [74]:
"""
!git clone https://github.com/openai/openai-quickstart-node.git
!pip install openai
!pip install tiktoken
!pip install python-dotenv
import os
os.chdir("openai-quickstart-node")
"""

'\n!git clone https://github.com/openai/openai-quickstart-node.git\n!pip install openai\n!pip install tiktoken\n!pip install python-dotenv\nimport os\nos.chdir("openai-quickstart-node")\n'

In [32]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [33]:
import os
import openai
import sys
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY2')

openai.api_key  = OPENAI_API_KEY

In [77]:

#prompt = f"""Based on the provided job description and CV, write a concise, compelling and personalized cover letter that highlights relevant skills and experiences.

#    Job Description:\n{job_description_text}\n\nCV:\n{cv_text}\n\n

#    Tailor the letter to the specific job requirements and showcase the candidate's match for the position. Be sure to mention specific accomplishments and quantify results whenever possible. Keep the tone professional and precise based on the cv.
#    As concise as possible no more than 4 sentences. Avoid using adjectives unless neccessary , as less as possible , do not use emotive adjectives in any case, use only factual information based on the cv_text , finalize with best regards, Name where the name of the applicant is taken from the CV"""


In [34]:
import openai

def generate_improved_text_critique(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a highly experienced career coach and recruiter. Your job is to critically evaluate cover letters based on job descriptions, resumes (CVs), and the expected professional standards for the role. You provide detailed feedback on various aspects of the cover letter."},
            {"role": "user", "content": prompt}
        ]

    )
    return response['choices'][0]['message']['content'].strip()

# Function to create the prompt
def create_critique_prompt(cover_letter_pdf, cv_text, job_description):
    prompt = f"""
    I need you to critique a cover letter that was submitted for an AI Developer role. The cover letter is provided as a PDF file, and I'll summarize its content below. The candidate's resume (CV) and the job description are also provided. Please evaluate the cover letter based on the following criteria:

    1. **Relevance to the Job**: Does the cover letter effectively address the specific responsibilities and requirements of the job as described in the job description?
    2. **Form and Structure**: Is the cover letter well-organized and professionally formatted? Does it follow standard cover letter conventions?
    3. **Reliability**: Does the cover letter convey a sense of reliability and trustworthiness? Is it free from errors and inconsistencies?
    4. **Professional Matching**: Does the cover letter demonstrate that the candidate's skills, experience, and background match the job's requirements?
    5. **Overall Impression**: How compelling is the cover letter in making the case that the candidate is a strong fit for the role?

    After critiquing the cover letter on these criteria, please provide an overall grade on a scale of 1-10, where 10 is the best.

    Here are the details:

    **Cover Letter Summary**:
    (Provide a summary of the content of the cover letter extracted from the PDF.)

    **Resume (CV) Text**:
    {cv_text}

    **Job Description**:
    {job_description}

    Please provide a detailed critique with specific points for improvement and an overall grade in the following format #Overall Grade : NUMBER#  where number is on the scale of 1-10. it is essential to keep this format in tact,
    I repeat keep this format in tact.
    """
    return generate_improved_text_critique(prompt)

In [35]:
import openai
import re

def generate_improved_text_critique(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a highly experienced career coach and recruiter. Your job is to critically evaluate cover letters based on job descriptions, resumes (CVs), and the expected professional standards for the role. You provide detailed feedback on various aspects of the cover letter."},
            {"role": "user", "content": prompt}
        ]
    )
    content = response['choices'][0]['message']['content'].strip()

    # Extract the overall grade using regex to ensure consistency
    match = re.search(r"#Overall Grade\s*:\s*(\d{1,2})#", content)
    if match:
        grade = match.group(1)
    else:
        grade = "N/A"  # Default in case the grade is missing or incorrectly formatted

    return content, grade

# Function to create the prompt
def create_critique_prompt(cover_letter_summary, cv_text, job_description):
    prompt = f"""
    I need you to critique a cover letter that was submitted for an AI Developer role. The cover letter is provided as a PDF file, and I'll summarize its content below. The candidate's resume (CV) and the job description are also provided. Please evaluate the cover letter based on the following criteria:

    1. **Relevance to the Job**: Does the cover letter effectively address the specific responsibilities and requirements of the job as described in the job description?
    2. **Form and Structure**: Is the cover letter well-organized and professionally formatted? Does it follow standard cover letter conventions?
    3. **Reliability**: Does the cover letter convey a sense of reliability and trustworthiness? Is it free from errors and inconsistencies?
    4. **Professional Matching**: Does the cover letter demonstrate that the candidate's skills, experience, and background match the job's requirements?
    5. **Overall Impression**: How compelling is the cover letter in making the case that the candidate is a strong fit for the role?

    After critiquing the cover letter on these criteria, please provide an overall grade on a scale of 1-10, where 10 is the best.

    Here are the details:

    **Cover Letter Summary**:
    {cover_letter_summary}

    **Resume (CV) Text**:
    {cv_text}

    **Job Description**:
    {job_description}

    Please provide a detailed critique with specific points for improvement and an overall grade in the following format:

    **#Overall Grade : NUMBER#**

    It is essential to keep this format intact. Please ensure that the grade is clearly provided using this exact format at the end of your response.
    """
    return generate_improved_text_critique(prompt)



In [57]:
import openai
import re

class GradeExtractionError(Exception):
    """Custom exception for errors in extracting the grade."""
    pass

def generate_improved_text_critique(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a highly experienced career coach and recruiter. Your job is to critically evaluate cover letters based on job descriptions, resumes (CVs), and the expected professional standards for the role. You provide detailed feedback on various aspects of the cover letter."},
            {"role": "user", "content": prompt}
        ]
    )
    content = response['choices'][0]['message']['content'].strip()

    # Attempt to extract the grade using regex
    match = re.search(r"Overall Grade.*?(\d+(\.\d+)?)", content, re.IGNORECASE)
    if match:
        grade = match.group(1)
    else:
        # If grade is not found, raise an exception
        raise GradeExtractionError("Failed to extract overall grade from the critique. Please review the critique content.")

    return content, grade

# Function to create the prompt
def create_critique_prompt(cover_letter_summary, cv_text, job_description):
    prompt = f"""
    I need you to critique a cover letter that was submitted for an AI Developer role. The cover letter is provided as a PDF file, and I'll summarize its content below. The candidate's resume (CV) and the job description are also provided. Please evaluate the cover letter based on the following criteria:

    1. **Relevance to the Job**: Does the cover letter effectively address the specific responsibilities and requirements of the job as described in the job description?
    2. **Form and Structure**: Is the cover letter well-organized and professionally formatted? Does it follow standard cover letter conventions?
    3. **Reliability**: Does the cover letter convey a sense of reliability and trustworthiness? Is it free from errors and inconsistencies?
    4. **Professional Matching**: Does the cover letter demonstrate that the candidate's skills, experience, and background match the job's requirements?
    5. **Overall Impression**: How compelling is the cover letter in making the case that the candidate is a strong fit for the role?

    After critiquing the cover letter on these criteria, please provide an overall grade on a scale of 1-10, where 10 is the best.

    Here are the details:

    **Cover Letter Summary**:
    {cover_letter_summary}

    **Resume (CV) Text**:
    {cv_text}

    **Job Description**:
    {job_description}

    Please provide a detailed critique with specific points for improvement and an overall grade in the following format:

    **#Overall Grade : NUMBER#**

    It is essential to keep this format intact. Please ensure that the grade is clearly provided using this exact format at the end of your response.
    """
    return generate_improved_text_critique(prompt)


In [36]:
import openai
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# Set your OpenAI API key

openai.api_key = OPENAI_API_KEY

# Function to generate improved text using OpenAI API
def generate_improved_text(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

# Function to improve the cover letter
def improve_cover_letter(CV_text, cover_letter,job_description_text, critique = ""):
    if critique == "":
      critique = create_critique_prompt(cover_letter, cv_text, job_description_text)
    #prompt = f"Given the following cover letter and critique, generate an improved version of the cover letter:\n\nCover Letter:\n{cover_letter}\n\nCritique:\n{critique}\n\nImproved Cover Letter:"
    prompt = f"""Based on the provided job description, original cover letter (the one that the critique refers to), and CV, write a concise, exact, factual and personalized cover letter that highlights relevant skills and experiences. the resulting letter should take into account the critique and improve accordingly


   Job Description:\n{job_description_text}\n\nCV:\n{CV_text}\n\nCritique:\n\ncover_letter:\n{cover_letter}\n\nCritique:\n{critique}\n\n

    Tailor the letter to the specific job requirements and showcase the candidate's match for the position. Be sure to mention specific accomplishments and quantify results whenever possible. Keep the tone professional and precise based on the cv.
   As concise as possible no more than 4 sentences. Avoid using adjectives unless neccessary , as less as possible , do not use emotive adjectives in any case, use only factual information based on the cv_text , finalize with best regards, Name where the name of the applicant is taken from the CV"""

    return generate_improved_text(prompt)

# Function to improve the CV
def improve_cv(cv_text, critique):
    prompt = f"Given the following CV and critique, generate an improved version of the CV:\n\nCV:\n{cv_text}\n\nCritique:\n{critique}\n\nImproved CV:"
    return generate_improved_text(prompt)

# Function to create a PDF from text
def create_pdf(output_pdf, title, name, text):
    print(output_pdf)
    pdf = SimpleDocTemplate(output_pdf, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Define custom styles for title and body text
    title_style = ParagraphStyle(name='Title', fontSize=16, leading=20, spaceAfter=20, alignment=1)
    name_style = ParagraphStyle(name='Name', fontSize=12, leading=14, spaceAfter=14, alignment=1)
    body_style = styles['BodyText']
    bold_style = ParagraphStyle(name='Bold', parent=styles['BodyText'], fontName='Helvetica-Bold')

    # Add title
    story.append(Paragraph(title, title_style))

    # Add name
    story.append(Paragraph(name, name_style))
    story.append(Spacer(1, 12))

    # Process and add the main text
    paragraphs = text.split("\n\n")
    for para in paragraphs:
        story.append(Paragraph(para.replace('\n', '<br/>'), body_style))
        story.append(Spacer(1, 12))

    # Build the PDF
    pdf.build(story)



In [37]:
%%time
#generate_cover_letter
#extract_key_phrases
#create_pdf_with_bold_strings(output_pdf, title, cv_text, text, bold_strings)
#improve_cover_letter
#improved_cover_letter
#extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3)

cv_text = load_and_extract_text("CoverLetter/Data/CV_GPT_rev.pdf")
cover_letter_text = generate_cover_letter(model,cv_text, job_description).text
bold_strings = extract_strings(extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3))
create_pdf_with_bold_strings(output_pdf, title, cv_text, text, bold_strings)
improved_cover_letter = improve_cover_letter(cv_text,cover_letter_text,job_description)
#improved_cv = improve_cv(original_cv, cv_critique)

#job_description

HERE generate_cover_letter
after generate_cover_letter
HERE key phrases
6
CPU times: user 322 ms, sys: 18.1 ms, total: 340 ms
Wall time: 18.8 s


In [58]:
%%time
critique,grade = create_critique_prompt( improved_cover_letter , cv_text, job_description)
critique

CPU times: user 87.2 ms, sys: 10.5 ms, total: 97.7 ms
Wall time: 14.3 s


"**Relevance to the Job**:\nThe cover letter effectively addresses some key responsibilities and requirements of the job. The candidate's experience in AI, machine learning, data standardization, and NLP is clearly defined and directly relevant to the key requirements in the job description. Further, his work in unified financial data formats is a particularly good match for the job's focus on creating a unified data format across various financial systems.\n\n**Form and Structure**:\nThe cover letter is concise and straightforward, which is generally favorable. However, it lacks the traditional structure of a cover letter, which generally includes an introductory statement, a body paragraph or two detailing relevant experience or skills, and a closing paragraph expressing interest in the role and eagerness for further discussion.\n\n**Reliability**:\nThe cover letter is relatively free from errors. It does, however, lack some context and detail which would help in understanding the ca

In [59]:
grade =float(grade)
grade

7.0

In [64]:
%%time
import re
d_grades = {}

for ii in range(10):

  print(grade)
  if grade < 9:
    improved_cover_letter = improve_cover_letter(cv_text,cover_letter_text,job_description, critique)
    critique,grade = create_critique_prompt( improved_cover_letter , cv_text, job_description)
    grade = float(grade)
    d_grades[(grade,ii)] = {"cover_letter": improved_cover_letter, "critique" : critique}
  else:
    break


5.0
8.5
7.5
7.0
6.0
7.2
6.5
8.5
6.0
7.0
CPU times: user 858 ms, sys: 136 ms, total: 994 ms
Wall time: 2min 49s


In [65]:
d_best = d_grades[max(d_grades.keys(), key = lambda x: x[0])]

In [66]:
d_best["cover_letter"]

"Dear Hiring Manager,\n\nI am writing to apply for the AI Developer position at your company, leveraging my Ph.D. in Mathematics and over 13 years of industry experience, including the development of predictive models, handling unified data integration, and applying NLP techniques for financial processes. My extensive work with financial systems and machine learning frameworks like TensorFlow, PyTorch, and scikit-learn, along with experience in ensuring data privacy and security, aligns well with the role's requirements. I have a proven track record of leading AI initiatives, as demonstrated by my work at Finzor Ltd. and Palo Alto Networks.\n\nBest regards,\nItay Ben-Dan"

In [67]:
d_best["critique"]

"1. **Relevance to the Job**: The cover letter demonstrates a high degree of relevance to the AI Developer role. The candidate directly refers to his extensive experience in developing predictive models, handling unified data integration, and applying NLP techniques—all of which are mentioned in the job description—and aligns these competencies with the job requirements. His mention of data privacy and security also matches the job requirements.\n   \n2. **Form and Structure**: The cover letter adheres to professional standards in terms of form and structure. It is concise and directly addresses the position in question. However, some elaboration on the candidate's experience and skills would have been beneficial. For example, he could have provided some specifics about his projects and results achieved, which would give the hiring manager a clearer idea of his capabilities.\n\n3. **Reliability**: The cover letter displays a high degree of reliability. It is free of errors and mistakes

In [47]:
grade

'N/A'

In [56]:
critique

"### Cover Letter Critique for AI Developer Role\n\n#### 1. **Relevance to the Job**\n\n**Strengths**:\n- The candidate highlights their extensive experience in AI, particularly in finance, which aligns well with the job description.\n- Mentioning specific experience with Python, TensorFlow, and NLP techniques directly addresses key technical requirements of the role.\n\n**Areas for Improvement**:\n- The cover letter can be enhanced by explicitly connecting the candidate’s past projects to the responsibilities listed in the job description (e.g., automating invoice reconciliation and standardizing financial data formats).\n- More details on how the candidate has collaborated with finance and accounting teams would further demonstrate their suitability for integrating AI solutions into existing workflows.\n\n#### 2. **Form and Structure**\n\n**Strengths**:\n- The letter is concise, which is often appreciated by hiring managers.\n- The candidate clearly states their background and expert

In [84]:
improved_cover_letter

'Dear Hiring Manager,\n\nI am writing to apply for the AI Developer position at your company. My experience includes developing and deploying AI and machine learning models for financial domains, as demonstrated by my work at Finzor Ltd. and Palo Alto Networks. I have successfully implemented NLP solutions to extract key information from unstructured data, unified diverse data formats, and ensured data privacy and security best practices in all projects. With strong programming skills in Python and experience with TensorFlow, PyTorch, and scikit-learn, I am confident in my ability to meet your needs for automating invoice reconciliation and standardizing financial data.\n\nBest regards,  \nItay Ben-Dan'

In [85]:
float(re.findall("[0-9\.]+",re.findall("Grade[\*].: [\*0-9\.]+",critique)[-1] )[0])

TypeError: expected string or bytes-like object

In [ ]:
re.findall("Overall Grade ?: [\*0-9\.]+",critique)

In [86]:
critique

("**Relevance to the Job**: The cover letter does a good job of addressing the duties and requirements of the AI Developer role, particularly in relation to developing AI models, working with financial data, and implementing NLP solutions. The candidate also alludes to their experience with data privacy and security, a requirement in the job description. However, they fail to mention their business requirement handling abilities and their capacity to collaborate with non-technical teams, both of which were key components in the job description. So, there's room for improvement in this area.\n\n**Form and Structure**: The cover letter is brief and it adheres to a basic structure - starting with the applicant's declaration of intent, followed by a summary of relevant experience, then ending with a note of confidence in their suitability for the role. However, it lacks a personal touch or any notable details making the candidate stand out. The letter could benefit from a more engaging and

In [ ]:
grade = int(re.findall("[0-9]+",re.findall("Grade: \*\*[0-9]+\*\*",critique)[0] )[0])
grade

In [ ]:
cover_letter_text

In [ ]:
improved_cover_letter

In [ ]:
job_description

In [ ]:

name = improved_cover_letter.split("\n")[-1]
target_pdf_file = "cover_letter_improved2.pdf"
title = ("Cover Letter")
create_pdf(target_pdf_file, title, name,improved_cover_letter )

In [ ]:


# Example usage

critique_prompt = create_critique_prompt(cover_letter_text, original_cv, job_description)
response = generate_improved_text(critique_prompt)

print(response)


In [ ]:
!ls